In [1]:
import pandas as pd
import numpy as np

Подгружаю вспомогательные файлы из таблицы

In [19]:
diseases = pd.read_csv('../../../project/diseases.csv')
diseases.head(6)

,original,dataset,unified,disease_id
0,Heart Disease,eg,Заболевание сердца,1.0
1,Viral Fever or Cold,eg,Простуда,2.0
2,Jaundice,eg,Желтуха,3.0
3,Food poisoning,eg,Пищевое отравление,4.0
4,Patient is normal,eg,-,NaN
5,influenza,OHAS,Грипп,5.0


In [20]:
symptoms = pd.read_csv('../../../project/symptoms.csv')
symptoms.head(6)

,original,dataset,unified,symptom_id
0,L.A. Pain,eg,Боль в животе,1.0
1,U.A. pain,eg,Боль в животе,1.0
2,Vomiting feeling,eg,Рвота,2.0
3,Yellowship Urine,eg,Неестественная желтизна мочи,3.0
4,Indigestion,eg,Несварение,4.0
5,Temperature,eg,Повышенная температура,5.0


Итоговый датасет. Столбцы - айдишники симптомов, в последнем будем айди болезни.

In [30]:
data = pd.DataFrame(columns = list(map(int,symptoms['symptom_id'].dropna().unique().tolist())) + ['disease_id'])
#data.columns = symptoms['symptom_id'].unique().tolist() + ['disease_id']
data.head()

,1,2,3,4,5,6,7,8,9,10,...,265,266,267,268,269,270,271,272,273,disease_id


## EG dataset

In [63]:
eg_data = pd.read_csv('../../../project/raw_data/eg/data.csv')
eg_data.head()

,Disease,Temperature,Pulse Rate,L.A Pain,U.A Pain,Vomiting Feeling,Yellowish Urine,Indigestion
0,0,10,13.8,0,10,0,0,0
1,1,11,13.6,10,10,0,0,0
2,2,13,14.0,10,0,0,10,0
3,3,11,12.8,0,0,10,0,10
4,0,14,15.0,0,10,0,0,0


Небольшая обработка

0: Heart Disease
1: Viral Fever or Cold
2: Jaundice
3: Food Poisoning
4: Patient is normal

In [64]:
eg_data.loc[eg_data.Disease == 0, 'Disease'] = 'Heart Disease '
eg_data.loc[eg_data.Disease == 1, 'Disease'] = 'Viral Fever or Cold '
eg_data.loc[eg_data.Disease == 2, 'Disease'] = 'Jaundice'
eg_data.loc[eg_data.Disease == 3, 'Disease'] = 'Food poisoning'
eg_data.loc[eg_data.Disease == 4, 'Disease'] = 'Patient is normal'

eg_data.loc[eg_data.Temperature > 10, 'Temperature'] = 1
eg_data.loc[eg_data.Temperature == 10, 'Temperature'] = 0
eg_data.drop('Pulse Rate', axis = 1, inplace = True)

In [65]:
for column in eg_data.columns[2:]:
    eg_data.loc[eg_data[column] == 10, column] = 1

In [66]:
eg_data.head()

,Disease,Temperature,L.A Pain,U.A Pain,Vomiting Feeling,Yellowish Urine,Indigestion
0,Heart Disease,0,0,1,0,0,0
1,Viral Fever or Cold,1,1,1,0,0,0
2,Jaundice,1,1,0,0,1,0
3,Food poisoning,1,0,0,1,0,1
4,Heart Disease,1,0,1,0,0,0


Теперь итерационная обработка и добавление в итоговый набор

In [94]:
for index, record in eg_data.iterrows():
    #создаю новую запись, заполняю все нулями
    new_row = dict.fromkeys(data.columns, 0)
    #если пациент в норме, он не обрабатывается
    if record.Disease == 'Patient is normal':
        continue
    #беру из diseases нужный айдишник по original названию и вставляю в поле diseases_id
    new_row['disease_id'] = int(diseases.loc[(diseases.dataset == 'eg') & (diseases.original == record.Disease), 
                                                'disease_id'].values[0])
    #вручную симптомы
    new_row[symptoms.loc[(symptoms.dataset == 'eg') & (symptoms.original == 'Temperature'), 'symptom_id'].values[0]] = record.Temperature
    new_row[symptoms.loc[(symptoms.dataset == 'eg') & (symptoms.original == 'L.A. Pain'), 'symptom_id'].values[0]] = record['L.A Pain']
    new_row[symptoms.loc[(symptoms.dataset == 'eg') & (symptoms.original == 'U.A. pain'), 'symptom_id'].values[0]] = record['U.A Pain']
    new_row[symptoms.loc[(symptoms.dataset == 'eg') & (symptoms.original == 'Vomiting feeling'), 'symptom_id'].values[0]] = record['Vomiting Feeling']
    new_row[symptoms.loc[(symptoms.dataset == 'eg') & (symptoms.original == 'Yellowship Urine'), 'symptom_id'].values[0]] = record['Yellowish Urine']
    new_row[symptoms.loc[(symptoms.dataset == 'eg') & (symptoms.original == 'Indigestion'), 'symptom_id'].values[0]] = record['Indigestion']
    #вставляю новую строку в итоговый набор
    data = data.append(new_row, ignore_index=True)

In [95]:
data.head()

,1,2,3,4,5,6,7,8,9,10,...,265,266,267,268,269,270,271,272,273,disease_id
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3,0,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
4,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
